# MODELE XGBOOST TARGET CALIBRE + SOUMISSION (sur Colab)

## PREPROCESSING

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import panel as pn

data = pd.read_csv('/content/gdrive/My Drive/Defi-IA/result_requetes.csv')
#data = pd.read_csv('result_requetes.csv')

#data.dtypes
#data.head()

In [3]:
#On affecte le bon type aux variables qualitatives
data["city"]=pd.Categorical(data["city"],ordered=False)
data["language"]=pd.Categorical(data["language"],ordered=False)
data["group"]=pd.Categorical(data["group"],ordered=False)
data["brand"]=pd.Categorical(data["brand"],ordered=False)

data["mobile"]=pd.Categorical(data["mobile"],ordered=False)
data["parking"]=pd.Categorical(data["parking"],ordered=False)
data["pool"]=pd.Categorical(data["pool"],ordered=False)
data["children_policy"]=pd.Categorical(data["children_policy"],ordered=False)

data["hotel_id"]=pd.Categorical(data["hotel_id"],ordered=False)
data["avatar_id"]=pd.Categorical(data["avatar_id"],ordered=False)

data["price"]=pd.DataFrame(data["price"], dtype=float)

#data.dtypes
#data.head()

In [4]:
X = data[["nb_requete","date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]

In [5]:
Y = data[["price"]]

## APPRENTISSAGE DE MODELE

In [6]:
!pip install category_encoders > /dev/null 2>&1

In [7]:
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder, MEstimateEncoder
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [8]:
numerical_transformer = StandardScaler()
#categorical_transformer = TargetEncoder()
#categorical_transformer = MEstimateEncoder()
categorical_transformer = OneHotEncoder()

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("Standard_scaler", numerical_transformer, selector(dtype_exclude="category")),
        ("Cat_encoder", categorical_transformer, selector(dtype_include="category")),
    ]
)

In [ ]:
#n_estimators + grand? 100 par défaut
#max_features?
#max_depth? 3 par défaut
#cv? 5 par défaut

In [32]:
pip = Pipeline(steps=[('Preprocessor', preprocessor),
                      #('Boosting', HistGradientBoostingRegressor()),
                      ('Boosting', GradientBoostingRegressor()),
                      #('Boosting', GradientBoostingRegressor(n_estimators=300, max_depth=10)),
                      ]
               )

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [34]:
y_train = np.ravel(y_train)

In [35]:
pip

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5e5bec6a0>),
                                                 ('cat-encoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5e5bec700>)])),
                ('Boosting', GradientBoostingRegressor())])

In [19]:
pip.fit(X_train, y_train)

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5e5bec6a0>),
                                                 ('cat-encoder',
                                                  OneHotEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5e5bec700>)])),
                ('Boosting',
                 GradientBoostingRegressor(max_depth=10, n_estimators=300))])

In [20]:
print(f"pip score: {pip.score(X_test, y_test):.2f}")

pip score: 0.99


In [60]:
param_grid = {
    "Boosting__n_estimators":[100,300,500],
    "Boosting__max_features":[1.,0.5,0.2],
    "Boosting__max_depth":[7,10,13],
}

grid_search = GridSearchCV(pip, param_grid, cv=10)

In [61]:
grid_search

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('Preprocessor',
                                        ColumnTransformer(transformers=[('standard_scaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5e5bec6a0>),
                                                                        ('cat-encoder',
                                                                         OneHotEncoder(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7fa5e5bec700>)])),
                                       ('Boosting',
                                        GradientBoostingRegressor())]),
             param_grid={'Boosting__max_depth': [7, 10, 13],
                         'Boosting__max_features': [1.0, 

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print("Best params:")
print(grid_search.best_params_)

In [ ]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

In [ ]:
print(f"GridSearch score: {grid_search.score(X_test, y_test):.3f}")

In [ ]:
# POUR ENREGISTRER/CHARGER LE MODELE
#from joblib import dump, load
#dump(xgb, 'XGB_model_saved.joblib')
#xgb_loaded = load('XGB_model_saved.joblib')

## TEST DU MODELE SUR LE FICHIER DEFI

In [ ]:
test = pd.read_csv('/content/gdrive/My Drive/Defi-IA/test_set3.csv')
#test.head()

In [ ]:
#On affecte le bon type aux variables qualitatives
test["city"]=pd.Categorical(test["city"],ordered=False)
test["language"]=pd.Categorical(test["language"],ordered=False)
test["group"]=pd.Categorical(test["group"],ordered=False)
test["brand"]=pd.Categorical(test["brand"],ordered=False)

test["mobile"]=pd.Categorical(test["mobile"],ordered=False)
test["parking"]=pd.Categorical(test["parking"],ordered=False)
test["pool"]=pd.Categorical(test["pool"],ordered=False)
test["children_policy"]=pd.Categorical(test["children_policy"],ordered=False)

test["hotel_id"]=pd.Categorical(test["hotel_id"],ordered=False)
test["avatar_id"]=pd.Categorical(test["avatar_id"],ordered=False)

#test.dtypes
#test.head()

In [ ]:
T = test[["nb_requete","date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]

In [ ]:
#Yprev=pip.predict(T)
Yprev=grid_search.predict(T)

In [ ]:
Yprev0=np.around(Yprev, decimals=1)

In [ ]:
Yindice = pd.DataFrame(np.arange(0,len(Yprev0)),columns = ['index'])

In [ ]:
Yprice=pd.DataFrame(Yprev0,columns = ['price'])
Ysub=Yindice.join(Yprice, on=None, how='right', lsuffix='', rsuffix='', sort=False)
Ysub.to_csv('/content/gdrive/My Drive/Defi-IA/sample_submissionXGB30.csv', index= False)
#Ysub.to_csv('./submission/sample_submissionXGB.csv', index= False)